In [93]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome import options
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from time import sleep

In [127]:
options = Options()
options.headless = False
navegador = webdriver.Chrome(options=options)
link=''
navegador.get(url=link)
sleep(3)
inputUsuario = navegador.find_element(by=By.ID,value='txtUser')
inputUsuario.send_keys('')
inputSenha = navegador.find_element(by=By.ID,value='txtPassword')
inputSenha.send_keys('')

sleep(2)
botao = navegador.find_element(by=By.NAME,value='sign-in')
botao.click()
sleep(2)

rps = navegador.find_element(by=By.LINK_TEXT,value= 'RPS')
rps.click()
sleep(2)

#select_element = navegador.find_element(by=By.NAME,value='empresa')
select_element = Select(navegador.find_element(by=By.NAME,value='empresa'))
select = Select(select_element)
select.select_by_visible_text('AGE S.A.')

C:\Users\guilherme\AppData\Local\Temp\ipykernel_31380\2829102187.py:2: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="empresa"]"}
  (Session info: chrome=109.0.5414.75)
Stacktrace:
Backtrace:
	(No symbol) [0x00BB6643]
	(No symbol) [0x00B4BE21]
	(No symbol) [0x00A4DA9D]
	(No symbol) [0x00A81342]
	(No symbol) [0x00A8147B]
	(No symbol) [0x00AB8DC2]
	(No symbol) [0x00A9FDC4]
	(No symbol) [0x00AB6B09]
	(No symbol) [0x00A9FB76]
	(No symbol) [0x00A749C1]
	(No symbol) [0x00A75E5D]
	GetHandleVerifier [0x00E2A142+2497106]
	GetHandleVerifier [0x00E585D3+2686691]
	GetHandleVerifier [0x00E5BB9C+2700460]
	GetHandleVerifier [0x00C63B10+635936]
	(No symbol) [0x00B54A1F]
	(No symbol) [0x00B5A418]
	(No symbol) [0x00B5A505]
	(No symbol) [0x00B6508B]
	BaseThreadInitThunk [0x755600F9+25]
	RtlGetAppContainerNamedObjectPath [0x76FB7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x76FB7B8E+238]
